In [1]:
import cogent3
from cogent3 import load_unaligned_seqs
from cogent3 import get_app
from cogent3 import make_aligned_seqs
from os import listdir
from os.path import isfile, join

In [2]:
concat_alns_app = get_app("concat", moltype="dna")
codon_align = get_app("progressive_align", "nucleotide", guide_tree="(Human:0.06,Chimpanzee:0.06,Gorilla:0.1)")
ffold = get_app("take_codon_positions", fourfold_degenerate=True)

In [3]:
#store all the alignments on onlyfiles list
folder = '/xdisk/masel/uliseshmc/EstimatingUd/primates10_112/aligns_ortho/'
onlyfiles = [f for f in listdir(folder) if isfile(join(folder, f))]    

#I want to reach the next line. Before I have to construct an auxilliary function that removes orthologs where one of the species is missing
#for i in range(0, len(onlyfiles)):
for i in range(0, 5):
    #Read alignments
    aln = load_unaligned_seqs(folder + onlyfiles[i], moltype='dna')
    aln = aln.rename_seqs(lambda x: x.split("-")[0])

    #Rename alignments rows
    hcm = aln.take_seqs(["homo_sapiens", "pan_troglodytes", "gorilla_gorilla"]).degap()
    name_map = {
        "homo_sapiens": "Human",
        "pan_troglodytes": "Chimpanzee",
        "gorilla_gorilla": "Gorilla"
    }
    #auxilliary function to rename sequences
    def renamer(name):
        return name_map.get(name, name)

    hcm = hcm.rename_seqs(renamer)
    hcm

    #Doing nucleotide model meanwhile. I got to change this for codon eventually
    aln_codonaligned = codon_align(hcm)

    #Extract four fold sites
    #Do I need omit_gap_pos() or degap()
    aln_ffold = ffold(aln_codonaligned).omit_gap_pos()

    if i==0:
        concat_aln = aln_ffold
    else:
        concat_aln = concat_alns_app([concat_aln, aln_ffold])

In [4]:
#Get distances of concatenated sequences 
sm = get_app("model", "GN", time_het="max", optimise_motif_probs=True, show_progress=True)
result = sm(concat_aln)
lengthstree = result.lf.get_lengths_as_ens()
lengthstree

   0%|          |00:00<?

   0%|          |00:00<?

{'Chimpanzee': np.float64(9.999983869577135e-07),
 'Gorilla': np.float64(1.0000069447833995e-06),
 'Human': np.float64(9.999983428411142e-07)}